In [1]:
from fastai.vision.all import *
import torchvision 
import numpy as np
import torch
import cv2
import os
from modules_th.video_block import *
import random
import time

In [2]:
mean = [0.43216, 0.394666, 0.37645]
std = [0.22803, 0.22145, 0.216989]

model = create_body(torchvision.models.video.r2plus1d_18, cut=-2)

In [71]:
def create_video_generator(vid_path, skip=2):
    vidcap = cv2.VideoCapture(str(vid_path))
    duration = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
    height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width =  int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
    l = int(duration//skip)
    frame = np.zeros((height,width,3), dtype=np.uint8)
    for frame_pos in range(0, l*skip, skip):
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame_pos)
        res, frame[:] = vidcap.read()
        yield frame         
    vidcap.release()

In [72]:
vid_paths = get_files('/mnt/data/eugeniomarinelli/UCF_Crimes/Videos', extensions='.mp4')
vid_path = random.choice(vid_paths)
for frame in create_video_generator(vid_path):
    print(frame.__array_interface__['data'][0])

94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
94175458341728
9417545834

KeyboardInterrupt: 

In [73]:
def get_and_save_embs(b, path):
    embs = model(b)
    for emb in embs.unbind():
         torch.save(emb, path) ### REMEMBER TO PUT PATH!!

bs, size, l, skip = 16, 224, 50, 2
pl = Pipeline([PILImage.create,
               Resize(size,method=ResizeMethod.Pad), 
               ToTensor(), 
               IntToFloatTensor(), 
               Normalize.from_stats(mean,std)])

shape = bs, l, 3, size, size
fr_ts = torch.zeros(*shape[2:], dtype=float)
snip = torch.zeros(*shape[1:], dtype=float)
b = torch.zeros(*shape, dtype=float)

for vid in vid_paths:
    for i, frame in enumerate(create_video_generator(vid_path, skip=2)):
        n_fr, n_snip = i%l, (i//l)%bs 
        fr_ts[:] = pl(frame)
        snip[n_fr] = fr_ts
        if n_fr==l-1: b[n_snip] = snip
        if n_snip==bs-1: get_and_save_embs(b, path) ### REMEMBER TO CHANGE PATH IN THE FUNCTION!!

KeyboardInterrupt: 

In [ ]:
files = get_files('/mnt/data/eugeniomarinelli/UCF_Crimes/Videos',extensions='.pt')
files.map(os.remove)

(#76600) [None,None,None,None,None,None,None,None,None,None...]